## youtube8Mのデータセット準備

とりあえずローカルでDatasetAPIで学習を開始できるようにするのが目的

### ローカルへのデータダウンロード

youtube8mのデータは以下のコマンドで取って来れます。
下のセルを実行すると全データの1/100のトレーニングデータを取得します。

環境変数からオプションを渡すような作りになってるみたい。
このpartition=2の部分がよくわからないけど、とりあえずこのままでw

In [1]:
# youtube8m のディレクトリに移動
import os
while os.getcwd().split('/')[-1] != 'youtube8m': os.chdir('..')
os.getcwd()

'/home/jupyter/ASLOpenProject/youtube8m'

In [2]:
BUCKET="asl-mixi-project-bucket"

In [3]:
%%bash 
CURRENT_DIR=$(PWD)
mkdir ./data/video/
cd ./data/video/
curl data.yt8m.org/download.py > download.py

shard=1,100 partition=2/video/train mirror=us python download.py
cd $CURRENT_DIR

Resuming Download ...
Files remaining 0
All done. No more files to download.


bash: line 1: PWD: command not found
mkdir: cannot create directory ‘./data/video/’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4450  100  4450    0     0  35098      0 --:--:-- --:--:-- --:--:-- 35317


In [4]:
!ls ./data/video/*.tfrecord -l | wc -l

41


In [5]:
!ls ./data/video/*.tfrecord -alh | head

-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0093.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0111.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0208.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.7M Dec  6 08:34 ./data/video/train0274.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.4M Dec  6 08:34 ./data/video/train0276.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0352.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0434.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0477.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.5M Dec  6 08:34 ./data/video/train0503.tfrecord
-rw-r--r-- 1 jupyter jupyter 4.6M Dec  6 08:34 ./data/video/train0580.tfrecord


### storageへのコピー

必要があれば実行。ローカルで実行する分には必要ありません。

In [ ]:
import os
os.environ['BUCKET'] = BUCKET

!gsutil cp ./data/video/*.tfrecord gs://$BUCKET/data/youtube-8m/

### データセットの作成用のコード

In [6]:
import tensorflow as tf

print(tf.__version__)

1.15.0


In [7]:
tf.enable_eager_execution()

In [8]:
from data import read_dataset

In [9]:
ds = read_dataset("./data/video/*.tfrecord", tf.estimator.ModeKeys.TRAIN, batch_size=2)

In [10]:
for row in ds.take(3):
    print('-----')
    print(row)
    print('mean_rgb:', row[0]['mean_rgb'])
    print('mean_audio:', row[0]['mean_audio'])
    

-----
({'id': <tf.Tensor: id=57, shape=(2,), dtype=string, numpy=array([b'V98D', b'oz8D'], dtype=object)>, 'mean_rgb': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fa466a84908>, 'mean_audio': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fa466a84080>}, <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fa466a84278>)
mean_rgb: SparseTensor(indices=tf.Tensor(
[[   0    0]
 [   0    1]
 [   0    2]
 ...
 [   1 1021]
 [   1 1022]
 [   1 1023]], shape=(2048, 2), dtype=int64), values=tf.Tensor(
[-0.8320534   0.8792135   1.6785632  ... -0.01001757 -0.09127247
 -0.19412214], shape=(2048,), dtype=float32), dense_shape=tf.Tensor([   2 1024], shape=(2,), dtype=int64))
mean_audio: SparseTensor(indices=tf.Tensor(
[[  0   0]
 [  0   1]
 [  0   2]
 [  0   3]
 [  0   4]
 [  0   5]
 [  0   6]
 [  0   7]
 [  0   8]
 [  0   9]
 [  0  10]
 [  0  11]
 [  0  12]
 [  0  13]
 [  0  14]
 [  0  15]
 [  0  16]
 [  0  17]
 [  0  18]
 [  0  19]
 [  0